In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.cluster import pair_confusion_matrix
import scipy
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from sklearn import metrics
from scipy.stats.stats import pearsonr 
from sklearn import metrics
import itertools
import networkx as nx
from scipy.sparse import csgraph
import umap
from importlib import reload
from matplotlib import rc
from scipy import stats

from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

rc('text', usetex=True)

# locals
import utils as ut
reload(ut)

sc.settings.verbosity = 3
import gget

# locals
import utils as ut
reload(ut)

2022-12-06 11:46:25.780338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 11:46:25.892510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2022-12-06 11:46:25.892535: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 11:46:25.918544: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 11:46:26.6862

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [2]:
clusterPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/clusters.csv"
cf = pd.read_parquet(clusterPath)
print(cf.shape)
print(cf['cellType'].unique())
cf.head()

(427882, 7)
['Macrophages' 'B cells' 'Monocytes' 'NK cells' 'Dendritic cells'
 'T cells' 'Stromal cells']


,gene,scores,pvals,logfoldchanges,clusterId,key,cellType
0,H2-EB1,36.343353,4.575346e-181,3.564091,1,ND,Macrophages
1,H2-AA,35.311523,4.761517e-183,3.430534,1,ND,Macrophages
2,H2-AB1,34.411163,2.815112e-177,3.234597,1,ND,Macrophages
3,CD74,33.910587,2.451588e-177,3.302062,1,ND,Macrophages
4,H2-DMA,31.695400,2.991793e-123,2.667091,1,ND,Macrophages


In [3]:
"""load scRNA"""
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/"

rna = {}

for f in os.listdir(dirPath):
    
    if f.endswith('.h5ad'):
        key = f.split(".")[0]
        fullPath = f"{dirPath}{f}"
        data = sc.read(fullPath)
        rna[key] = data
    
print("done")

done


In [6]:
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/imgenProfiles/"

for f in os.listdir(dirPath):
    fullPath = f"{dirPath}{f}"
    
    t = pd.read_csv(fullPath)
    print(f)
    print(t.columns)
    print()

GSE122597_Normalized_Gene_count_table.csv
Index(['gene_symbol', 'B.fo.Sp#1.1', 'B.fo.Sp#1.10', 'B.fo.Sp#1.11',
       'B.fo.Sp#1.12', 'B.fo.Sp#1.13', 'B.fo.Sp#1.14', 'B.fo.Sp#1.15',
       'B.fo.Sp#1.16', 'B.fo.Sp#1.2', 'B.fo.Sp#1.3', 'B.fo.Sp#1.4',
       'B.fo.Sp#1.5', 'B.fo.Sp#1.6', 'B.fo.Sp#1.8', 'B.fo.Sp#1.9', 'MF.PC#1.1',
       'MF.PC#1.10', 'MF.PC#1.11', 'MF.PC#1.12', 'MF.PC#1.13', 'MF.PC#1.14',
       'MF.PC#1.15', 'MF.PC#1.16', 'MF.PC#1.2', 'MF.PC#1.3', 'MF.PC#1.4',
       'MF.PC#1.5', 'MF.PC#1.6', 'MF.PC#1.7', 'MF.PC#1.8', 'MF.PC#1.9',
       'MF.PC#2.1', 'MF.PC#2.2', 'MF.PC#2.3', 'MF.PC#2.4', 'MF.PC#2.5',
       'MF.PC#2.6', 'MF.PC#2.7', 'MF.PC#2.8', 'NK.Sp#1.1', 'NK.Sp#1.10',
       'NK.Sp#1.11', 'NK.Sp#1.12', 'NK.Sp#1.13', 'NK.Sp#1.14', 'NK.Sp#1.15',
       'NK.Sp#1.16', 'NK.Sp#1.17', 'NK.Sp#1.18', 'NK.Sp#1.2', 'NK.Sp#1.3',
       'NK.Sp#1.4', 'NK.Sp#1.5', 'NK.Sp#1.6', 'NK.Sp#1.7', 'NK.Sp#1.8',
       'NK.Sp#1.9', 'T.4.Nve.#2.1', 'T.4.Nve.#2.2', 'T.4.Nve.Sp#1.1',
       '